In [2]:
import sys
!{sys.executable} -m pip install pycbc ligo-common emcee==2.2.1
#Install the PyCBC 

In [3]:
from pycbc.inference import models
for model_name in models.models:
    print(model_name)
from pycbc.inference import models
from pycbc.distributions import Uniform
from pycbc.inference import sampler
import pylab, numpy
from numpy.random import uniform, seed
from pycbc.distributions import Uniform
from pycbc.distributions import JointDistribution
import pylab, numpy
from scipy import stats
from pycbc.catalog import Merger
from pycbc.psd import interpolate, inverse_spectrum_truncation
from pycbc.frame import read_frame
from pycbc.filter import highpass, resample_to_delta_t
from astropy.utils.data import download_file
from pycbc.inference import models, sampler
from pycbc.distributions import Uniform, JointDistribution, SinAngle
import numpy
import copy
from matplotlib import animation
import pylab
from matplotlib.animation import PillowWriter
from IPython.display import Image
from IPython.display import HTML
# All this is defined in this module
import lalsimulation
# LAL is used for constants
import lal
import numpy as np
import os

/home/idies/miniconda3/envs/py38/lib/python3.8/site-packages/pycbc/population/fgmc_functions.py:21: UserWarning: matplotlib.pyplot as already been imported, this call will have no effect.
  use('Agg')


test_eggbox
test_normal
test_rosenbrock
test_volcano
test_posterior
test_prior
gaussian_noise
marginalized_phase
marginalized_polarization
marginalized_hmpolphase
brute_parallel_gaussian_marginalize
gated_gaussian_noise
gated_gaussian_margpol
single_template
relative


In [4]:
#import the models from lalsimulation VV (and write the names)
modelnames=lalsimulation.SimNeutronStarEOSNames
modelnames=['ALF1', 'ALF2', 'ALF3', 'ALF4', 'AP1', 'AP2', 'AP3', 'AP4',
       'APR4_EPP', 'BBB2', 'BGN1H1', 'BPAL12', 'BSK19', 'BSK20', 'BSK21',
       'ENG', 'FPS', 'GNH3', 'GS2', 'H1', 'H2', 'H3', 'H4', 'H5',
       'H6', 'H7', 'MPA1', 'MS1B', 'MS1B_PP', 'MS1_PP', 'MS1', 'MS2',
       'PAL6', 'PCL2', 'PS', 'QMC700', 'SLY4', 'SLY', 'SQM1', 'SQM2',
       'SQM3', 'WFF1', 'WFF2', 'WFF3', 'APR', 'BHF_BBB2', 'KDE0V',
       'KDE0V1', 'RS', 'SK255', 'SK272', 'SKA', 'SKB', 'SKI2', 'SKI3',
       'SKI4', 'SKI5', 'SKI6', 'SKMP', 'SKOP', 'SLY2', 'SLY230A', 'SLY9',
       'HQC18']

In [5]:
#VV The code used for this function was supplied by the supervisor for this project, Ian Harry 
def lambdafunct(mass,EOS): #define the lambda function and imputs
    #
    #To use one of these EOSs, we must first load it into to an eos object
    my_intermediate_eos_object = lalsimulation.SimNeutronStarEOSByName(EOS)
    #
    #And then for some reason we create a *second* object from this
    my_eos_object = lalsimulation.CreateSimNeutronStarFamily(my_intermediate_eos_object)
    #
    # Then I can do things with that object. As this stuff is actually coded in C, it's not very pythonic
    # to use. But C is not an object-oriented language, it's functional.
    love_number = lalsimulation.SimNeutronStarLoveNumberK2(mass * lal.MSUN_SI, my_eos_object)
    #
    # Yes, it is actually called this. Lambda is a rescaling of the "love number"
    # This function takes the mass in kg, so multiply mass in solar masses by lal.MSUN_SI
    # which is 1.988E30
    radius = lalsimulation.SimNeutronStarRadius(mass * lal.MSUN_SI, my_eos_object)
    rescale_constant = lal.G_SI * mass * lal.MSUN_SI  / (lal.C_SI*lal.C_SI * radius)
    # As lambda is a built-in function in python, don't use it as a variable name.
    lamda = 2./3. * love_number * rescale_constant**(-5.)
    return lamda

In [25]:
#the cell is to constrain the number of equations by tidal deformation

#"snapped" is the number of models removed
snapped=0 

#for each EOS we consider in the model names, we test whether it produce tidal deformation between
#0 or 1000 for a binary 1.36 solar mass merger and remove it from the list if it does not.

#for our list of equations of state
for EOS in modelnames:   
    #calculate the tidal deformation (lam) due to a 1.36 solar mass merger for that equation of state
    lam=lambdafunct(1.36,EOS)
    #if the tidal deformation is outside our set range, remove it from our list of considered EOS
    if lam >= 1000 or lam <= 0:
        #c will be printed to show if in or out
        c='Outsider'
        #remove the EOS from our list
        modelnames.remove(EOS)
        #increase the count of snapped models
        snapped=snapped+1
    else:
        #other leave it in the code (and say so)
        c='Inside'
    print(EOS,lam,c)
#print new list of model names
print(modelnames)
#print the nubmer of models snapped
print('number of models snapped',snapped)

BHF_BBB2 267.3488835790448 Under
APR4_EPP 297.3444973295465 Under
MS2 272.1310627623031 Under
WFF2 279.2253254596244 Under
['BHF_BBB2', 'APR4_EPP', 'MS2', 'WFF2']
number of models snapped 0


In [ ]:
#the injection file for the signal we are measuring
#the parameters are model after 170817 and are taken from : ***
#LAMMM is designed to be replaced by the tidal deformation due to our equation of state
injection_config = """
[static_params]
f_lower=40
ra = 2.2
dec = -1.25
inclination = 2.5
coa_phase = 1.5
polarization = 1.75
f_ref = 40
distance = 40
mass1 = 1.36
mass2 = 1.36
lambda1 = LAMMM
lambda2 = LAMMM
tc = 0
approximant = TaylorF2
taper = start

[variable_params]
"""
#The EOS we will use for the injectoipon will be BHF_BBB2, given by EOSCORRECT and will this will decide 
#the tidal deformation that takes place for this run, change this if you want to measure
#EOS model comparison for another EOS. 
EOSCORRECT = 'BHF_BBB2'
#lam3 is the lambda due to the equation of state for our injection EOS, in this case BHF_BBB2
lam3=str(lambdafunct(1.36,EOSCORRECT))
#replace LAMMM with lam3 to produce a config file with the appropriate equation of state
injection = injection_config.replace("""LAMMM""", lam3)
#write the injection to a config file
with open('injection.ini', 'w') as output_file:
    output_file.write(injection)
    
#write the bash command to run in a python script to produce injection.hdf file
bashCommand = "pycbc_create_injections --verbose \
        --config-files injection.ini \
        --ninjections 1 \
        --seed 10 \
        --output-file injection.hdf \
        --variable-params-section variable_params \
        --static-params-section static_params \
        --dist-section prior \
        --force"
#run the bash command
os.system(bashCommand)

#this next section focuses of producing conversion files between mass, EOS and tidal deformnation used in the inference
#Masslistbase is the mass values used for this run. 
masslistbase=np.linspace(1.26,1.46,num=201)
config_list = []            #defunct
injection_list = []         #defucnt
save_to_list =[]            #'defunct'
for EOS in modelnames:      #for the EOS we are considering
    print('Started ACS file for',EOS)     #'allows us to read progress for the loop
    lambdatable = ['yeet','yeeten']       #first row in the array, neccessary for stacking, remove at the end
    masslist = masslistbase               #reset the mass list (defunct) 
    for mass in masslist:                 #run thru the mass list for each EOS
        lamlist = []                      #empty list for the columns to have the mass and lambda values added later
        try:                              #the code crashes if the masses are to heavy for the EOS to support 
            lam = lambdafunct(mass,EOS)   #But if it runs the EOS produces Neutron stars with tidal deformation
        except Exception:                 #If it doesn't assume black hole
            lam = 0 #because code failures with various equations of state denote that the star collapses into a black hole and ergo has no detectable tidal deformation
        lamlist.append(mass) #add mass as first collumn
        lamlist.append(lam) #add mass as second collum
        lambdatable = np.vstack([lambdatable, lamlist]) # stack the columns
    to_save = lambdatable[1:] #remove the first row so PyCBC can read
    #save the text file to the name ACS so that the INference can read it. This file gets overwritted by each new equation of state for each loop to save space
    np.savetxt('ACS', to_save, fmt='%s', delimiter=' ', newline='\n', header='', footer='', comments='# ', encoding=None) # save the text file with name of the EOS used to generate the lambdas
    ##  myfile.close() 
    print('Done with mass row', EOS)    #give a show of progress
    lam=lambdafunct(1.36,EOS)        #produce the tidal deformation (lam) as a function of the EOS we are considering
    #write the model config we are writing to a string
    #much of this code is from *** but may have undergone alterations to fit this work
    #replace EinsteinTelecopeP1600143 with another PSD and rerun to compare the code, don't forget to change the 
    #name of the output files to stop overwriting (more on this below)
    model_config = """
    [model]
    name = relative
    low-frequency-cutoff = 40.0
    high-frequency-cutoff = 1024.0
    mass1_ref = 1.36
    mass2_ref = 1.36
    lambda1_ref= LAMMM 
    lambda2_ref= LAMMM
    tc_ref = 0

    [data]
    instruments = H1
    trigger-time = 0
    analysis-start-time = -50
    analysis-end-time = 5
    ; strain settings
    sample-rate = 4096
    fake-strain = H1:EinsteinTelescopeP1600143
    fake-strain-seed = H1:44
    ; psd settings
    psd-estimation = median-mean
    psd-inverse-length = 8
    psd-segment-length = 8
    psd-segment-stride = 4
    psd-start-time = -128
    psd-end-time = 128
    ; even though we're making fake strain, the strain
    ; module requires a channel to be provided, so we'll
    ; just make one up
    channel-name = H1:STRAIN
    ; Providing an injection file will cause a simulated
    ; signal to be added to the data
    injection-file = injection.hdf
    ; We'll use a high-pass filter so as not to get numerical errors from the large
    ; amplitude low frequency noise. Here we use 15 Hz, which is safely below the
    ; low frequency cutoff of our likelihood integral (20 Hz)
    strain-high-pass = 15
    ; The pad-data argument is for the high-pass filter: 8s are added to the
    ; beginning/end of the analysis/psd times when the data is loaded. After the
    ; high pass filter is applied, the additional time is discarded. This pad is
    ; *in addition to* the time added to the analysis start/end time for the PSD
    ; inverse length. Since it is discarded before the data is transformed for the
    ; likelihood integral, it has little affect on the run time.
    pad-data = 8

    [sampler]
    name = emcee_pt
    nwalkers = 200
    ntemps = 20
    effective-nsamples = 2000
    checkpoint-interval = 400
    max-samples-per-chain = 1000

    [sampler-burn_in]
    burn-in-test = nacl & max_posterior

    [variable_params]
    ; waveform parameters that will vary in MCMC
    mass1=
    mass2=
    
    [static_params]
    f_lower=40
    ra = 2.2
    dec = -1.25
    inclination = 2.5
    coa_phase = 1.5
    polarization = 1.75
    f_ref = 40
    distance = 40
    tc = 0
    approximant = TaylorF2
    taper = start

    [prior-mass1]
    name=uniform
    min-mass1=1.26
    max-mass1=1.46

    [prior-mass2]
    name=uniform
    min-mass2=1.26
    max-mass2=1.46
    
    [waveform_transforms-lambda1]
    name = lambda_from_tov_file
    mass_param = mass1
    lambda_param = lambda1
    mass_lambda_file = ACS

    [waveform_transforms-lambda2]
    name = lambda_from_tov_file
    mass_param = mass2
    lambda_param = lambda2
    mass_lambda_file = ACS
    """
    lam2 = str(lam) #a string is needed to write replace the values
    #replace the values in the model for relative binning to start at
    #the model we are considering
    config = model_config.replace("""LAMMM""", lam2) 
    #with the values replaced we can write the string to a config file so we can run it with Bayesian Inference
    with open('CONFIG', 'w') as output_file:
        output_file.write(config)
    #'show that we have reach the inference (the lengthiest section of this loop)
    print('Reached Inference for',EOS)

    #write the inference bash command to the sting
    bashCommand2 ="pycbc_inference \
            --config-files CONFIG \
            --nprocesses=4 \
            --output-file INF \
            --seed 0 \
            --force \
            --verbose"
    #the output files must remain, replace the output file name with something related to the EOS and if running for different
    #PSDs, change the 'ET' to reflect that another detector PSD is being tested
    bashCommand2 = bashCommand2.replace("INF", f"Output_{EOS}_ET.hdf")
    #run the command
    os.system(bashCommand2)
    #show that the inference is completed for the EOS and that the next one is staring
    print('Completed Inference for',EOS)
    print('--------------------------------------------------------------------------------------------------')
#for reading these files and extracting the bayesian evidence and the evidence error, see part II 